In [1]:
library(rnoaa)
library(tidyverse)
library(lubridate)
library(rlang)
library(stringr)

Warning message:
"package 'rnoaa' was built under R version 4.1.2"
Registered S3 method overwritten by 'httr':
  method           from  
  print.cache_info hoardr

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.0.2     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 4.1.2"
Warning message:
"package 'tidyr' was built under R version 4.1.2"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union



Attaching package: 'rlang'


The following objects are masked from 'package:purrr':

    %@%, as_function, 

In [2]:
stations <- ghcnd_stations()

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/ghcnd-stations.rds

date created (size, mb): 2022-02-12 20:25:20 (2.159)

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/ghcnd-inventory.rds

date created (size, mb): 2022-02-12 20:26:28 (2.669)



In [77]:
# check climate var we can use for a specific station(city)

names(ghcnd_search(stationid = "USC00186350", date_min = "1950-01-01", date_max = "2022-01-31"))
names(ghcnd_search(stationid = "GME00127786", date_min = "1950-01-01", date_max = "2022-01-31"))
names(ghcnd_search(stationid = "JA000047759", date_min = "1950-01-01", date_max = "2022-01-31"))
names(ghcnd_search(stationid = "CA001108395", date_min = "1950-01-01", date_max = "2022-01-31"))

In [5]:
# load df. don't change df names as they will be used as input of function

japan <- read.csv("../data/raw/japan.csv")
kyoto <- read.csv("../data/raw/kyoto.csv")
liestal <- read.csv("../data/raw/liestal.csv")
washingtondc <- read.csv("../data/raw/washingtondc.csv")

kyoto<- kyoto|> 
  mutate(year = strtoi(substr(year, 1, 4)))
meteoswiss <- read.csv("../data/raw/meteoswiss.csv")
south_korea <- read.csv("../data/raw/south_korea.csv")

In [ ]:
# retrieve climate parameter by city and month

retrieve_climate <- function(stationid, para, month_list = seq(1, 12)){
    para <- enquo(para)
    df <- ghcnd_search(stationid = stationid, date_min = "1950-01-01", date_max = "2022-01-31")[[as_name(para)]]
    result <- df |> 
      mutate(year = year(date),
             month = month(date),
             !!para := !!para / 10) |> 
    filter(month %in% month_list) |> 
    drop_na() |> 
    group_by(year) |>
    summarise(!!para := mean(!!para, na.rm = TRUE))
}

In [ ]:
# join the climate and bloom df together and plot
# create month list for further use

month = c("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec")

plot_ts_bday_climate <- function(stationid, city_df, para, month_list = seq(1, 12)) {
    # environment(retrieve_climate) <-  environment()
    para <- enquo(para)
    df <- retrieve_climate(stationid, !!para, month_list)
    city_cli <- merge(city_df, df, by = "year")
    a <- ""
    for (num in month_list){
        a <- paste(a, month[num], sep = " ")
    }
    out <- city_cli |> 
          ggplot(aes(x = year)) +
          geom_line(aes(y = bloom_doy), color = "blue") +
          geom_line(aes(y = !!para / 0.25), color = "red") +
          scale_y_continuous(
            name = "Bloom Day",
            sec.axis = sec_axis(~. * 0.25, name = paste("Climate parameter:", as_name(para), sep = " "))
          ) +
          labs(title = paste("Time series of bloom day and climate variable of month:\n", a, sep = ""), 
               x = "Year") +
          theme(
            axis.title.y = element_text(color = "blue", size = 20),
            axis.title.y.right = element_text(color = "red", size = 20),
            axis.title.x = element_text(size = 20)
          )
    out
}

In [ ]:
plot_ts_bday_climate("JA000047759", kyoto, tmin)